Link Colab: https://colab.research.google.com/drive/1bieSuj5kAgW4P6HLhgbDUUydYXHhNLkq#scrollTo=Bi7ViWf18KYo

Import Library Yang dibutuhkan

In [ ]:
import pandas as pd
import gdown
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score,StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import time
import joblib
from imblearn.over_sampling import RandomOverSampler
from sklearn.ensemble import GradientBoostingClassifier
import gdown

In [ ]:
gdown.download('https://drive.google.com/uc?id=1arAGNjbj3A8pB_qyj17ze_kJhWJ0u6ni' , 'data_bersih.csv', quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=1arAGNjbj3A8pB_qyj17ze_kJhWJ0u6ni
From (redirected): https://drive.google.com/uc?id=1arAGNjbj3A8pB_qyj17ze_kJhWJ0u6ni&confirm=t&uuid=4eb50f51-a8a3-4c47-91d9-5bd865c29a4f
To: /content/data_bersih.csv
100%|██████████| 523M/523M [00:09<00:00, 56.8MB/s]


'data_bersih.csv'

import dataset

In [ ]:
df = pd.read_csv("data_bersih.csv")

Tampilkan Dataset

In [ ]:
df

,JT_clean,Role_clean,jb_clean,sk_clean
0,digital marketing specialist,social medium manager,social medium manager oversee organization soc...,social medium platform e g facebook twitter in...
1,web developer,frontend web developer,frontend web developer design implement user i...,html cs javascript frontend framework e g reac...
2,operation manager,quality control manager,quality control manager establish enforce qual...,quality control process methodology statistica...
3,network engineer,wireless network engineer,wireless network engineer design implement mai...,wireless network design architecture wi fi sta...
4,event manager,conference manager,conference manager coordinate manages conferen...,event planning conference logistics budget man...
...,...,...,...,...
1615935,mechanical engineer,mechanical design engineer,mechanical design engineer create develop mech...,mechanical engineering cad software e g solidw...
1615936,manager,director,director oversees organization department tech...,strategic planning leadership management budge...
1615937,mechanical engineer,mechanical design engineer,mechanical design engineer create develop mech...,mechanical engineering cad software e g solidw...
1615938,hr coordinator,training coordinator,training coordinator design implement employee...,training program coordination training materia...


Disini Kita hanya menggunakan 30% Sample dari dataset

In [ ]:
df = df.sample(n=7000, random_state=1)

In [ ]:
df.shape

(7000, 4)

In [ ]:
df

,JT_clean,Role_clean,jb_clean,sk_clean
1451761,human resource manager,talent acquisition manager,talent acquisition manager oversee recruitment...,talent sourcing interviewing onboarding recrui...
296010,ux ui designer,interaction designer,interaction designer specialize designing user...,interaction design principle user behavior psy...
106007,executive assistant,office manager,office manager oversee daily office operation ...,office administration management budgeting exp...
1145103,mechanical designer,hvac system designer,design optimize heating ventilation air condit...,hvac system design load calculation energy eff...
1245469,litigation attorney,corporate litigator,corporate litigator specialize corporate law h...,corporate law commercial litigation dispute re...
...,...,...,...,...
220216,supply chain manager,demand planner,demand planner analyzes historical sale data f...,demand forecasting inventory management data a...
1454340,investment analyst,risk analyst,risk analyst ass manages financial risk within...,risk assessment data analysis financial modeli...
764703,ux ui designer,interaction designer,interaction designer specialize designing user...,interaction design principle user behavior psy...
124421,product designer,industrial designer,industrial designer creates improves product d...,industrial design principle cad 3d modeling pr...


In [ ]:
df['JT_clean'].value_counts()

,count
JT_clean,
ux ui designer,221
digital marketing specialist,130
software engineer,112
network engineer,108
software tester,105
...,...
network analyst,13
event manager,11
market analyst,11


Rename

In [ ]:
df = df.rename(columns={'JT_clean': 'job title', 'Role_clean': 'role', 'jb_clean': 'job desc', 'sk_clean': 'skill'})

In [ ]:
df

,job title,role,job desc,skill
1451761,human resource manager,talent acquisition manager,talent acquisition manager oversee recruitment...,talent sourcing interviewing onboarding recrui...
296010,ux ui designer,interaction designer,interaction designer specialize designing user...,interaction design principle user behavior psy...
106007,executive assistant,office manager,office manager oversee daily office operation ...,office administration management budgeting exp...
1145103,mechanical designer,hvac system designer,design optimize heating ventilation air condit...,hvac system design load calculation energy eff...
1245469,litigation attorney,corporate litigator,corporate litigator specialize corporate law h...,corporate law commercial litigation dispute re...
...,...,...,...,...
220216,supply chain manager,demand planner,demand planner analyzes historical sale data f...,demand forecasting inventory management data a...
1454340,investment analyst,risk analyst,risk analyst ass manages financial risk within...,risk assessment data analysis financial modeli...
764703,ux ui designer,interaction designer,interaction designer specialize designing user...,interaction design principle user behavior psy...
124421,product designer,industrial designer,industrial designer creates improves product d...,industrial design principle cad 3d modeling pr...


Ekstraksi Fitur dan encode Label

In [ ]:
tfidf1 = TfidfVectorizer(max_features=5000)

X1 = tfidf1.fit_transform(df['role'])

label_encoder1 = LabelEncoder()
y1 = label_encoder1.fit_transform(df['skill'])

Split dataset sebesar 80:20

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X1, y1, test_size=0.2, random_state=42)

Model1 Prediksi Skill Berdasarkan Job

In [ ]:
lr = LogisticRegression(penalty='l2', C=1.0, max_iter=1000)
lr.fit(X_train, y_train)

y_pred = lr.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
accuracy_percentage = accuracy * 100
report = classification_report(y_test, y_pred)

print(f'Akurasi: {accuracy_percentage:.2f}%')
print("Classification Report:")
print(report)

Akurasi: 97.79%
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         6
           1       1.00      1.00      1.00         1
           2       1.00      1.00      1.00         3
           3       1.00      1.00      1.00         4
           4       1.00      1.00      1.00         4
           6       1.00      1.00      1.00         6
           7       1.00      1.00      1.00         3
           8       1.00      1.00      1.00         3
           9       1.00      1.00      1.00         2
          10       1.00      1.00      1.00         1
          11       1.00      1.00      1.00         3
          12       1.00      1.00      1.00         1
          13       1.00      1.00      1.00         2
          14       1.00      1.00      1.00         4
          16       1.00      1.00      1.00         4
          17       1.00      1.00      1.00         5
          18       0.00      0.00      0.0

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Alternatif Model Lain

In [ ]:
nb = MultinomialNB()
nb.fit(X_train, y_train)
y_pred_nb = nb.predict(X_test)

accuracy = accuracy_score(y_test, y_pred_nb)
accuracy_percentage = accuracy * 100
report = classification_report(y_test, y_pred_nb)

print(f'Akurasi: {accuracy_percentage:.2f}%')
print("Classification Report:")
print(report)

Akurasi: 76.79%
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         6
           1       0.00      0.00      0.00         1
           2       0.00      0.00      0.00         3
           3       0.17      1.00      0.30         4
           4       1.00      1.00      1.00         4
           6       1.00      1.00      1.00         6
           7       1.00      1.00      1.00         3
           8       1.00      1.00      1.00         3
           9       0.00      0.00      0.00         2
          10       1.00      1.00      1.00         1
          11       1.00      1.00      1.00         3
          12       0.00      0.00      0.00         1
          13       1.00      1.00      1.00         2
          14       0.00      0.00      0.00         4
          16       1.00      1.00      1.00         4
          17       1.00      1.00      1.00         5
          18       0.00      0.00      0.0

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
svm = SVC(kernel='linear')
svm.fit(X_train, y_train)
y_pred_svm = svm.predict(X_test)

accuracy = accuracy_score(y_test, y_pred_svm)
accuracy_percentage = accuracy * 100
report = classification_report(y_test, y_pred_svm)

print(f'Akurasi: {accuracy_percentage:.2f}%')
print("Classification Report:")
print(report)

Akurasi: 98.64%
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         6
           1       1.00      1.00      1.00         1
           2       1.00      1.00      1.00         3
           3       1.00      1.00      1.00         4
           4       1.00      1.00      1.00         4
           6       1.00      1.00      1.00         6
           7       1.00      1.00      1.00         3
           8       1.00      1.00      1.00         3
           9       1.00      1.00      1.00         2
          10       1.00      1.00      1.00         1
          11       1.00      1.00      1.00         3
          12       1.00      1.00      1.00         1
          13       1.00      1.00      1.00         2
          14       1.00      1.00      1.00         4
          16       1.00      1.00      1.00         4
          17       1.00      1.00      1.00         5
          18       1.00      1.00      1.0

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred_rf)
accuracy_percentage = accuracy * 100
report = classification_report(y_test, y_pred_rf)

print(f'Akurasi: {accuracy_percentage:.2f}%')
print("Classification Report:")
print(report)

Akurasi: 98.64%
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         6
           1       1.00      1.00      1.00         1
           2       1.00      1.00      1.00         3
           3       1.00      1.00      1.00         4
           4       1.00      1.00      1.00         4
           6       1.00      1.00      1.00         6
           7       1.00      1.00      1.00         3
           8       1.00      1.00      1.00         3
           9       1.00      1.00      1.00         2
          10       1.00      1.00      1.00         1
          11       1.00      1.00      1.00         3
          12       1.00      1.00      1.00         1
          13       1.00      1.00      1.00         2
          14       1.00      1.00      1.00         4
          16       1.00      1.00      1.00         4
          17       1.00      1.00      1.00         5
          18       1.00      1.00      1.0

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Testing

In [ ]:
def predictSkill(role):
    x1 = tfidf1.transform([role])
    # Logistic Regresion
    startTime = time.time()
    skillLogisticRegression = lr.predict(x1)
    timeLogisticRegression = time.time() - startTime
    skillPredictedLogisticRegression = label_encoder1.inverse_transform(skillLogisticRegression)[0]
    #Naive Bayes
    startTime = time.time()
    skillNaiveBayes = nb.predict(x1)
    timeNaiveBayes = time.time() - startTime
    skillPredictedNaiveBayes = label_encoder1.inverse_transform(skillNaiveBayes)[0]

    #SVM
    startTime = time.time()
    skillSVM = svm.predict(x1)
    timeSVM = time.time() - startTime
    skillPredictedSVM = label_encoder1.inverse_transform(skillSVM)[0]

    #Random Forest
    startTime = time.time()
    skillRandomForest = rf.predict(x1)
    timeRandomForest = time.time() - startTime
    skillPredictedRandomForest = label_encoder1.inverse_transform(skillRandomForest)[0]

    #print
    print(f'Role: {role}')
    print(f'Logistic Regression Predicted Skill: {skillPredictedLogisticRegression} (Time: {timeLogisticRegression:.6f} seconds)')
    print(f'Naive Bayes Predicted Skill: {skillPredictedNaiveBayes} (Time: {timeNaiveBayes:.6f} seconds)')
    print(f'SVM Predicted Skill: {skillPredictedSVM} (Time: {timeSVM:.6f} seconds)')
    print(f'Random Forest Predicted Skill: {skillPredictedRandomForest} (Time: {timeRandomForest:.6f} seconds)')


role = input("Masukkan Role: ")

predictSkill(role)


Masukkan Role: Data Science
Role: Data Science
Logistic Regression Predicted Skill: social science research method qualitative quantitative data analysis survey design implementation ethnographic research interviewing skill research ethic statistical software e g stata nvivo report writing communication skill critical thinking empathy (Time: 0.001086 seconds)
Naive Bayes Predicted Skill: data analysis tool e g sql python data visualization tool e g tableau power bi statistical analysis data cleansing transformation data modeling communication data insight problem solving attention detail business acumen (Time: 0.000838 seconds)
SVM Predicted Skill: social science research method qualitative quantitative data analysis survey design implementation ethnographic research interviewing skill research ethic statistical software e g stata nvivo report writing communication skill critical thinking empathy (Time: 0.005073 seconds)
Random Forest Predicted Skill: social science research method qua

Dikarenakan dari keempat model ini dapat disimpulkan, Disini kami lebih Memilih Untuk menggunakan Logistic Linear dikarenakan

In [ ]:
joblib.dump(lr, 'ModelLogreg1.pkl')
joblib.dump(tfidf1, 'tfidf1.pkl')
joblib.dump(label_encoder1, 'label_encoder1.pkl')

['label_encoder1.pkl']

---
Model Rekomendasi Pelatihan


---

Link Dataset:
 https://docs.google.com/spreadsheets/d/14ZoREh756KPLAocBGj2afbHWpff2rmo3XTjfCJQDVvQ/edit?gid=0#gid=0

In [ ]:
url = 'https://docs.google.com/spreadsheets/d/14ZoREh756KPLAocBGj2afbHWpff2rmo3XTjfCJQDVvQ/export?format=csv&gid=0'

Load dataset dari spreadsheet

In [ ]:
data = pd.read_csv(url)

In [ ]:
data.columns

Index(['Title', 'Role', 'Link '], dtype='object')

Cek distribusi sample, disini terlihat distribusi sample tidak maksimal maka akan kita resample

In [ ]:
print(data['Link '].value_counts())

Link 
https://www.coursera.org/learn/physical-therapy-exercise                              6
https://www.coursera.org/articles/research-analyst                                    5
https://www.udemy.com/course/ultimate-seo-training-course/                            5
https://www.coursera.org/professional-certificates/microsoft-power-bi-data-analyst    4
https://www.coursera.org/learn/brand                                                  4
                                                                                     ..
https://www.accounting.com/careers/financial-controller/how-to-become/                1
https://trafft.com/how-to-become-a-tax-consultant/                                    1
https://www.coursera.org/learn/accessibility                                          1
https://www.coursera.org/specializations/ui-ux-design                                 1
https://www.cgspectrum.com/career-pathways/environment-designer                       1
Name: count, Length: 308, 

In [ ]:
data.head()

,Title,Role,Link
0,Account Director,Account Strategist,https://www.coursera.org/learn/how-to-develop-...
1,Account Director,Business Development Director,https://www.insead.edu/executive-education
2,Account Executive,Sales Account Executive,https://www.coursera.org/learn/account-management
3,Account Executive,Key Account Executive,https://www.linkedin.com/learning/key-account-...
4,Account Executive,Advertising Account Executive,https://www.udemy.com/course/abm_masterclass/


Cek null

In [ ]:
data.isna().sum()

,0
Title,0
Role,0
Link,0


Buat fungsi untuk Preprocessing data

In [ ]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.tokenize import RegexpTokenizer
import re
from nltk.stem import WordNetLemmatizer
import nltk

nltk.download('stopwords')
nltk.download('wordnet')

def clean_text(text):

    text = re.sub(r'[^a-zA-Z0-9]', ' ', text.lower())

    tokenizer = RegexpTokenizer(r'\w+')
    words = tokenizer.tokenize(text)

    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]

    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]

    cleaned_text = ' '.join(words)

    return cleaned_text


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


Apply fungsi untuk preprocessing data

In [ ]:
data['Role'] = data['Role'].apply(clean_text)

In [ ]:
data

,Title,Role,Link
0,Account Director,account strategist,https://www.coursera.org/learn/how-to-develop-...
1,Account Director,business development director,https://www.insead.edu/executive-education
2,Account Executive,sale account executive,https://www.coursera.org/learn/account-management
3,Account Executive,key account executive,https://www.linkedin.com/learning/key-account-...
4,Account Executive,advertising account executive,https://www.udemy.com/course/abm_masterclass/
...,...,...,...
371,Web Developer,frontend web developer,https://www.coursera.org/professional-certific...
372,Web Developer,backend web developer,https://www.coursera.org/professional-certific...
373,Web Developer,full stack developer,https://www.coursera.org/professional-certific...
374,Wedding Planner,wedding consultant,https://www.udemy.com/course/wedding-planning/


In [ ]:
tfidfVector = TfidfVectorizer()

x = tfidfVector.fit_transform(data['Role'])

le = LabelEncoder()
y = le.fit_transform(data['Link '])

In [ ]:
ros = RandomOverSampler()

x_resampled, y_resampled = ros.fit_resample(x, y)

distribusi data Sebelum Di resampling

In [ ]:
pd.Series(y).value_counts()

,count
93,6
35,5
293,5
145,4
49,4
...,...
25,1
21,1
39,1
186,1


Distribusi data sesudah di re sampling

In [ ]:
pd.Series(y_resampled).value_counts()

,count
72,6
194,6
2,6
32,6
220,6
...,...
278,6
241,6
270,6
295,6


Split Dataset 80:20

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_resampled, y_resampled, test_size=0.2, random_state=42)

In [ ]:
logreg_link = LogisticRegression(max_iter=1000)
logreg_link.fit(x_train, y_train)

y_pred_lrlink = logreg_link.predict(x_test)

accuracy = accuracy_score(y_test, y_pred_lrlink)
accuracy_percentage = accuracy * 100
report = classification_report(y_test, y_pred_lrlink)

print(f'Akurasi: {accuracy_percentage:.2f}%')
print("Classification Report:")
print(report)

Akurasi: 75.14%
Classification Report:
              precision    recall  f1-score   support

           1       1.00      1.00      1.00         2
           3       1.00      1.00      1.00         1
           4       1.00      1.00      1.00         1
           6       1.00      1.00      1.00         1
           7       0.00      0.00      0.00         5
           8       1.00      1.00      1.00         2
           9       1.00      1.00      1.00         2
          10       1.00      1.00      1.00         3
          11       0.00      0.00      0.00         2
          12       1.00      1.00      1.00         2
          13       0.00      0.00      0.00         0
          14       1.00      1.00      1.00         1
          15       1.00      1.00      1.00         2
          17       0.00      0.00      0.00         2
          18       0.00      0.00      0.00         0
          19       1.00      1.00      1.00         1
          20       1.00      1.00      1.0

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.

In [ ]:
svm = SVC(kernel='linear', C=1.0)
svm.fit(x_train, y_train)

y_pred_svm = svm.predict(x_test)
accuracy_svm = accuracy_score(y_test, y_pred_svm)
accuracy_percentage_svm = accuracy_svm * 100
report_svm = classification_report(y_test, y_pred_svm)

print(f'Akurasi: {accuracy_percentage_svm:.2f}%')
print("Classification Report:")
print(report_svm)

Akurasi: 91.35%
Classification Report:
              precision    recall  f1-score   support

           1       1.00      1.00      1.00         2
           3       1.00      1.00      1.00         1
           4       1.00      1.00      1.00         1
           6       1.00      1.00      1.00         1
           7       0.00      0.00      0.00         5
           8       1.00      1.00      1.00         2
           9       1.00      1.00      1.00         2
          10       1.00      1.00      1.00         3
          11       0.00      0.00      0.00         2
          12       1.00      1.00      1.00         2
          14       1.00      1.00      1.00         1
          15       1.00      1.00      1.00         2
          17       1.00      1.00      1.00         2
          19       1.00      1.00      1.00         1
          20       1.00      1.00      1.00         3
          21       1.00      1.00      1.00         2
          22       1.00      1.00      1.0

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.

In [ ]:
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(x_train, y_train)

y_pred_rf = rf.predict(x_test)
accuracy_rf = accuracy_score(y_test, y_pred_rf)
accuracy_percentage_rf = accuracy_rf * 100
report_rf = classification_report(y_test, y_pred_rf)

print(f'Akurasi: {accuracy_percentage_rf:.2f}%')
print("Classification Report:")
print(report_rf)

Akurasi: 95.41%
Classification Report:
              precision    recall  f1-score   support

           1       1.00      1.00      1.00         2
           3       1.00      1.00      1.00         1
           4       1.00      1.00      1.00         1
           6       1.00      1.00      1.00         1
           7       1.00      1.00      1.00         5
           8       1.00      1.00      1.00         2
           9       1.00      1.00      1.00         2
          10       1.00      1.00      1.00         3
          11       0.00      0.00      0.00         2
          12       1.00      1.00      1.00         2
          14       1.00      1.00      1.00         1
          15       1.00      1.00      1.00         2
          17       1.00      1.00      1.00         2
          19       1.00      1.00      1.00         1
          20       1.00      1.00      1.00         3
          21       1.00      1.00      1.00         2
          22       1.00      1.00      1.0

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(n_estimators=100, random_state=42)
gb.fit(x_train, y_train)

y_pred_gb = gb.predict(x_test)
accuracy_gb = accuracy_score(y_test, y_pred_gb)
accuracy_percentage_gb = accuracy_gb * 100
report_gb = classification_report(y_test, y_pred_gb)

print(f'Gradient Boosting Akurasi: {accuracy_percentage_gb:.2f}%')
print("Gradient Boosting Classification Report:")
print(report_gb)

Gradient Boosting Akurasi: 95.41%
Gradient Boosting Classification Report:
              precision    recall  f1-score   support

           1       1.00      1.00      1.00         2
           3       1.00      1.00      1.00         1
           4       1.00      1.00      1.00         1
           6       1.00      1.00      1.00         1
           7       1.00      1.00      1.00         5
           8       1.00      1.00      1.00         2
           9       1.00      1.00      1.00         2
          10       1.00      1.00      1.00         3
          11       0.00      0.00      0.00         2
          12       1.00      1.00      1.00         2
          14       1.00      1.00      1.00         1
          15       1.00      1.00      1.00         2
          17       1.00      1.00      1.00         2
          19       1.00      1.00      1.00         1
          20       1.00      1.00      1.00         3
          21       1.00      1.00      1.00         2
      

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(x_train, y_train)

y_pred_knn = knn.predict(x_test)
accuracy_knn = accuracy_score(y_test, y_pred_knn)
accuracy_percentage_knn = accuracy_knn * 100
report_knn = classification_report(y_test, y_pred_knn)

print(f'KNN Akurasi: {accuracy_percentage_knn:.2f}%')
print("KNN Classification Report:")
print(report_knn)

KNN Akurasi: 81.89%
KNN Classification Report:
              precision    recall  f1-score   support

           1       1.00      1.00      1.00         2
           3       1.00      1.00      1.00         1
           4       1.00      1.00      1.00         1
           6       1.00      1.00      1.00         1
           7       0.00      0.00      0.00         5
           8       1.00      1.00      1.00         2
           9       1.00      1.00      1.00         2
          10       1.00      1.00      1.00         3
          11       0.00      0.00      0.00         2
          12       1.00      1.00      1.00         2
          14       1.00      1.00      1.00         1
          15       1.00      1.00      1.00         2
          17       1.00      1.00      1.00         2
          19       1.00      1.00      1.00         1
          20       1.00      1.00      1.00         3
          21       0.67      1.00      0.80         2
          22       1.00      1.00 

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.

Testing Model

In [ ]:
def predictSkill(role):
    x1 = tfidfVector.transform([role])

    # Logistic Regression
    startTime = time.time()
    skillLogisticRegression = logreg_link.predict(x1)
    timeLogisticRegression = time.time() - startTime
    skillPredictedLogisticRegression = le.inverse_transform(skillLogisticRegression)[0]

    # SVM
    startTime = time.time()
    skillSVM = svm.predict(x1)
    timeSVM = time.time() - startTime
    skillPredictedSVM = le.inverse_transform(skillSVM)[0]

    # Random Forest
    startTime = time.time()
    skillRandomForest = rf.predict(x1)
    timeRandomForest = time.time() - startTime
    skillPredictedRandomForest = le.inverse_transform(skillRandomForest)[0]

    # Gradient Boost
    startTime = time.time()
    skillGradientBoosting = gb.predict(x1)
    timeGradientBoosting = time.time() - startTime
    skillPredictedGradientBoosting = le.inverse_transform(skillGradientBoosting)[0]

    #KNN
    startTime = time.time()
    skillKNN = knn.predict(x1)
    timeKNN = time.time() - startTime
    skillPredictedKNN = le.inverse_transform(skillKNN)[0]

    print(f'Role: {role}')
    print(f'Logistic Regression Predicted Link: {skillPredictedLogisticRegression} \n (Waktu: {timeLogisticRegression:.6f} seconds)')
    print(f'SVM Predicted Link: {skillPredictedSVM} \n (Waktu: {timeSVM:.6f} seconds)')
    print(f'Random Forest Predicted Link: {skillPredictedRandomForest} \n (Waktu: {timeRandomForest:.6f} seconds)')
    print(f'Gradient Boosting Predicted Link: {skillPredictedGradientBoosting} \n (Waktu: {timeGradientBoosting:.6f} seconds)')
    print(f'KNN Predicted Link: {skillPredictedKNN} \n (Waktu: {timeKNN:.6f} seconds)')

In [ ]:
inp = input("Masukkan Role: ")
predictSkill(inp)

Masukkan Role: Data Analyst
Role: Data Analyst
Logistic Regression Predicted Link: https://www.udemy.com/course/data-quality-management-masterclass-the-complete-course/ 
 (Waktu: 0.001069 seconds)
SVM Predicted Link: https://www.coursera.org/professional-certificates/ibm-data-analyst 
 (Waktu: 0.002822 seconds)
Random Forest Predicted Link: https://www.coursera.org/professional-certificates/ibm-data-analyst 
 (Waktu: 0.007062 seconds)
Gradient Boosting Predicted Link: https://www.coursera.org/professional-certificates/ibm-data-analyst 
 (Waktu: 0.005284 seconds)
KNN Predicted Link: https://www.coursera.org/professional-certificates/ibm-data-analyst 
 (Waktu: 0.001693 seconds)


Setelah beberapa pertimbangan akhirnya kami memilih algoritma Gradient Boosting untuk mengklasifikasikan Link pembelajaran

In [ ]:
joblib.dump(gb, 'ModelLink.pkl')
joblib.dump(tfidfVector, 'tfidf.pkl')
joblib.dump(le, 'labelEncoder.pkl')

['labelEncoder.pkl']